In [4]:
! pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpcore-1.0.4-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 3.1 MB/s eta 0:00:00:00:01
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.4-py3-none-any.whl (77 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
sp

In [5]:
from openai import OpenAI

client = OpenAI(api_key="")

# Send request to OpenAI Models

In [203]:
import base64
import requests
from openai import OpenAI

# OpenAI API Key
api_key = ''

# Function to encode the image
def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')        

def gpt(text,
        model_name,
        image_path=list(),  
        system_prompt='You are a helpful assistant.', 
        temperature=1):
    
    # Prepare content for system and user-side
    system_content = [
            {
                "type": "text",
                "text": system_prompt
            }
          ]
    
    user_content = [
            {
                "type": "text",
                "text": text
            }
          ]
    
    # Add image to user content if the user input image paths
    if len(image_path) != 0:
        for i in range(len(image_path)):
            # Encode image
            base64_image = encode_image(image_path[i])
            image_dict = {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                    "detail": "low"
                }
            }
            user_content.append(image_dict)
    
    #print(user_content) #DEBUG

    
    # Request to server
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": model_name,
        "messages": [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": user_content
        }
        ],
        "max_tokens": 500,
        "temperature": temperature
    }
    
    # Get response
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    response_json = response.json()
    output = response_json['choices'][0]['message']['content']
    return output

## Test with GPT4-Vision

In [144]:
text = "Classify the activities and emotions in this image."

system_prompt_vision = """
You are a helpful assistant with the ability to analyze images and identify the activity and emotion of the person in the image.
When given an image, describe the activity and emotion. 
If no person is visible in the image, respond with 'unidentifiable' for both activity and emotion. 
Similarly, if the person's face is not clear, especially in surveillance footage, output 'unidentifiable' for the emotion while still attempting to identify the activity.
"""

output = gpt(text = text,
             model_name="gpt-4-vision-preview", 
             image_path = ['tmp/HACER_mid_frame/putonglasses_angry_7_j.png'], 
             system_prompt = system_prompt_vision,
             temperature = 0.2
             ) 
        
output 

"In the image, the person is sitting on a chair and appears to be adjusting or putting on eyeglasses with both hands. The individual's facial expression is not fully visible due to the hands covering the face, making it difficult to accurately determine the emotion. However, the posture and activity suggest a neutral or focused demeanor, as the person seems engaged in the task of handling the glasses."

## Test with GPT3.5 Turbo

### Intent detection

In [204]:
system_prompt_intent_detection = f"""
The current time is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.
You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object complete with the following keys:
1. 'patient_id': A string representing the ID of the patient the user is inquiring about
3. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question.
4. 'list_time':  A list of times for which data needs to be retrieved to answer the user's question. The system saves data in 30-minute period like 00:00:00 and 00:30:00.
5. 'vital_sign': A list of vital signs that the user is asking for. Here are the available vital sign: heart_rate, systolic_pressure, diastolic_pressure, respiratory_rate, body_temperature, oxygen_saturation
6. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly or if the user has requested a plot.
7. 'recognition': A Boolean value indicating whether the user is asking for activity or emotion recognition of the patient. Set to true if the user is asking for this information, otherwise false.
8. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient."""


#2. 'is_current': A Boolean value indicating whether the user is asking for data at the current moment. Set to true if the user is asking for this information, otherwise false.
text = "Give me the heart rate, blood pressure of the patient in the morning last 15 days?"           
output = gpt(system_prompt=system_prompt_intent_detection, text=text, model_name="gpt-3.5-turbo", temperature=0.2)
print(output)

{
    "patient_id": "unknown",
    "list_date": ["2024-02-28", "2024-02-29", "2024-03-01", "2024-03-02", "2024-03-03", "2024-03-04", "2024-03-05", "2024-03-06", "2024-03-07", "2024-03-08", "2024-03-09", "2024-03-10", "2024-03-11", "2024-03-12", "2024-03-13"],
    "list_time": ["06:00:00", "06:30:00", "07:00:00", "07:30:00", "08:00:00", "08:30:00"],
    "vital_sign": ["heart_rate", "systolic_pressure", "diastolic_pressure"],
    "is_plot": true,
    "recognition": false,
    "is_image": false
}


In [107]:
text = "Give me the current heart rate of the patient?"   
output = gpt(system_prompt=system_prompt_intent_detection, text=text, model_name="gpt-3.5-turbo", temperature=0.2)
print(output)

{
    "patient_id": "unknown",
    "list_date": [],
    "list_time": [],
    "vital_sign": ["heart rate"],
    "is_plot": false,
    "recognition": false,
    "is_image": false
}


In [197]:
text = "What is the patient name?"   
output = gpt(system_prompt=system_prompt_intent_detection, text=text, model_name="gpt-3.5-turbo", temperature=0.2)
print(output)

{
    "patient_id": "",
    "list_date": [],
    "list_time": [],
    "vital_sign": [],
    "is_plot": false,
    "recognition": false,
    "is_image": false
}


In [70]:
import json
data_dict = json.loads(output)
data_dict

{'patient_id': 'unknown',
 'list_date': ['current'],
 'time': [],
 'vital_sign': [],
 'is_plot': False,
 'recognition': True,
 'is_image': False}

## Language module

In [155]:
system_prompt_endpoint = """
You are a helpful medical assistant. Your task is to retrieve the information from the provided data 
to accurately answer questions from the user. The correctness of your answers is of utmost importance."""

text = """
The current time is 08:52:00 on March 14, 2024.

Patient information:
Name: Sarah White
Sex: Female
Address: 005 Boyd Wells, Lake Shannonborough, MH 77468
Caregiver phone number: +971561575483
Date of birth (yyyy-mm-dd): 1991-02-26
Age: 33

Image:
In the image, the person is sitting on a chair and appears to be adjusting or putting on eyeglasses with both hands. The individual's facial expression is not fully visible due to the hands covering the face, making it difficult to accurately determine the emotion. However, the posture and activity suggest a neutral or focused demeanor, as the person seems engaged in the task of handling the glasses.

Vital signs:
Timestamp (Year-Month-Day Hour:Minute:Second), Heart Rate, Systolic Pressure, Diastolic Pressure, Respiratory Rate, Body Temperature, Oxygen Saturation
2024-03-14 08:00:00, 57, 97, 75, 12, 36.8, 100
2024-03-14 07:00:00, 55, 98, 83, 14, 36.9, 95

Give me the vital of the patient now?
"""
output = gpt(system_prompt=system_prompt_endpoint, text=text, model_name="gpt-3.5-turbo", temperature=0.2)
print(output)

The vital signs of the patient Sarah White at the current time 08:52:00 on March 14, 2024, are as follows:

- Heart Rate: 57 bpm
- Systolic Pressure: 97 mmHg
- Diastolic Pressure: 75 mmHg
- Respiratory Rate: 12 breaths per minute
- Body Temperature: 36.8°C
- Oxygen Saturation: 100%


In [ ]:
08:30:00, March 13, 2024: heart_rate: 61, systolic_pressure: 99, diastolic_pressure: 75, respiratory_rate:	17, body_temperature: 37.8, oxygen_saturation: 99
08:30:00, March 12, 2024: heart_rate: 57, systolic_pressure: 97, diastolic_pressure: 75, respiratory_rate:	12, body_temperature: 36.8, oxygen_saturation: 100
08:30:00, March 11, 2024: heart_rate: 63, systolic_pressure: 98, diastolic_pressure: 80, respiratory_rate:	14, body_temperature: 36.3, oxygen_saturation: 93
08:30:00, March 10, 2024: heart_rate: 55, systolic_pressure: 98, diastolic_pressure: 83, respiratory_rate:	14, body_temperature: 36.9, oxygen_saturation: 95


In [358]:
patient_info = {
    "name": "Sarah White",
    "sex": "F",	
    "address": "005 Boyd Wells, Lake Shannonborough, MH 77468",	
    "phone": "+971561575483",
    "dob": "1991-02-26"	,
    "age": "31"
}

vital_sign = {
    "heart_rate": 61,
    "systolic_pressure": 99,
    "diastolic_pressure": 75,
    "respiratory_rate":	14,
    "body_temperature": 36.8,
    "oxygen_saturation": 99
}

system_prompt_intent_detection = f"""
The current time is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.
You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object complete with the following keys:
1. 'patient_id': A string representing the ID of the patient the user is inquiring about
3. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question. Leave the list empty if the user asks for data right now.
4. 'list_time':  A list of times for which data needs to be retrieved to answer the user's question. The system saves data in 30-minute period like 00:00:00 and 00:30:00. If the user asks for sessions during the day, please use the following information to fill in the list: Morning is from 5am to 12pm, Afternoon is from 12pm to 5pm, Evening is from 5pm to 9pm, Night is from 9pm to 4am. Leave the list empty if the user asks for data right now. 
5. 'vital_sign': A list of vital signs that the user is asking for. Here are the available vital sign: heart_rate, systolic_pressure, diastolic_pressure, respiratory_rate, body_temperature, oxygen_saturation
6. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly (when the number of data points is too large) or if the user has requested a plot.
7. 'recognition': A Boolean value indicating whether the user is asking for activity or emotion recognition of the patient. Set to true if the user is asking for this information, otherwise false.
8. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient.
"""

system_prompt_endpoint = """
You are a helpful medical assistant. Your task is to use the provided data to accurately and concisely answer user questions. 
The correctness of your answers is of utmost importance.
It's important to understand that blood pressure is measured as systolic pressure over diastolic pressure."""

system_prompt_vision = """
You are a helpful assistant with the ability to analyze images and identify the activity and emotion of the person in the image.
When given an image, describe the activity and emotion. 
If no person is visible in the image, respond with 'unidentifiable' for both activity and emotion. 
Similarly, if the person's face is not clear, especially in surveillance footage, output 'unidentifiable' for the emotion while still attempting to identify the activity.
"""

text_endpoint_format = """
The current time is {current_time}.

Patient information:
ID Number: {patient_id}
Name: {name}
Sex: {sex}
Address: {address}
Caregiver phone number: {phone}
Date of birth (yyyy-mm-dd): {dob}
Age: {age}

Image:
{image_description}

Vital signs:
{vital_signs_data}

{question}
"""



import re
import pandas as pd

patient_meta_df = pd.read_csv('tmp/fake_patient_meta_data.csv')
vital_sign_var_to_text = {
                'heart_rate': 'Heart Rate', 
                'systolic_pressure': 'Systolic Pressure', 
                'diastolic_pressure': 'Diastolic Pressure', 
                'respiratory_rate': 'Respiratory Rate', 
                'body_temperature': 'Body Temperature', 
                'oxygen_saturation': 'Oxygen Saturation'
            }

# check if the id_str is a valid patient id
def is_valid_id(id_str):
    if bool(re.match(r'^\d{5}$', id_str)): #check if the id_str follow id format
        if int(id_str) in patient_meta_df['patient_id'].values: # check if the id_str appears in the current database
            return True
        else: return False        
    else: return False

# extract patient_id from a text by find words matching the format of the id 
def extract_patient_id_from_text(text):
    # Regular expression pattern to match an ID like '00001'
    pattern = r"\d{5}"

    # Search for the pattern in the user response
    match = re.search(pattern, text)

    # Extract the ID if a match is found
    if match:
        patient_id = match.group(0)
        return patient_id
    
    else:
        return 'unknown'

# return the valid id to retrieve patient data
def get_id(id_from_intent_dict):
    patient_id = id_from_intent_dict
    while True:
        # check if the patient_id detected by 'intent detection' module is valid
        if is_valid_id(patient_id):
            break
            
        else: # ask user to provide the valid id
            temp = input('I did not receive a valid patient ID number. Please provide a valid ID number.')
            if temp == '':  # if the user cannot provide a valid patient_id, the session will stop
                return False
            patient_id = extract_patient_id_from_text(temp) 
            
    return patient_id
    
def combine_data_and_time(list_date, list_time):
    # combine date and time
    time_stamp_list = []
    for date in list_date:
        for time in list_time:
            time_stamp = f'{date} {time}'
            time_stamp_list.append(time_stamp)
            
    return time_stamp_list
    
def df_to_text(df, intent_dict, is_current):
    if not is_current: # user is not asking for current data
        # combine date and time
        time_stamp_list = combine_data_and_time(intent_dict['list_date'], intent_dict['list_time'])
   
    else: 
        time_stamp_list = [df['time_stamp'][0]] # if user ask for current data, df just has one row
    
    #print(time_stamp_list)
    
    # filter the df columns
    columns = ['time_stamp'] + intent_dict['vital_sign'] # list of columns to filter out
    data_df = df[columns]
    #print(data_df[:3])
    
    # get rows with time_stamp in time_stamp_list
    data_df = data_df[data_df['time_stamp'].isin(time_stamp_list)].reset_index()
    #print(data_df[:3])
    
    # convert the df into text to put into the endpoint prompt
    data_text = ''
    title_row = 'Timestamp (Year-Month-Day Hour:Minute:Second)'
    
    # 1. create the title row
    for vital_sign in intent_dict['vital_sign']:
        title_row += f', {vital_sign_var_to_text[vital_sign]}'
    
    data_text += f'{title_row} \n'  
    
    # 1. add data row into data_text
    for i in range(len(data_df)):
        data_row_text = data_df.iloc[i]['time_stamp']

        for vital_sign in intent_dict['vital_sign']:
            data_row_text += f', {data_df.iloc[i][vital_sign]}'
        
        data_text += f'{data_row_text} \n'
            
    return data_text


def NLP_engine(doctor_question):
    # initialize variables
    image_description = 'None'
    vital_signs_text = 'None' 
    show_list = [] # save image_path or plot_path that would be used to shown to the user
    
    ################################
    ##########   INTENT   ##########
    ################################
    
    intent = gpt(system_prompt=system_prompt_intent_detection, text=doctor_question, model_name="gpt-3.5-turbo", temperature=0.1)
    intent_dict = json.loads(intent)
    #print(intent_dict) #debug
    #print()
    
    # check the valid patient_id (did the user provide this information yet? does the id appear in the database?):
    patient_id =  get_id(intent_dict['patient_id'])
    
    if not patient_id: # if the user cannot provide a valid patient_id, the session will stop
        return "Exit", show_list
    else: 
        intent_dict['patient_id'] = patient_id # update the new valid patient_id into the intent_dict
       
    
    ################################
    ########## VITAL SIGN ##########
    ################################
    
    if len(intent_dict['vital_sign'])>0: # check if we need to get the vital sign
        ########## get real-time data from the EDGE DEVICE ##########
        if (len(intent_dict['list_date']) == 0) and (len(intent_dict['list_time']) == 0): 
            ############################# TODO: get data from EDGE DEVICE with values in keys of 'patient_id', 'vital_sign'
            vital_sign_df = pd.read_csv('tmp/2024_03.csv') 
            #############################
            
            vital_signs_text = df_to_text(vital_sign_df, intent_dict, is_current=True)
        
        ########## get real-time data from the AWS S3 ##########
        else: 
            # process some special case
            if len(intent_dict['list_date']) == 0: # the patient ask for data in some sessions in the current day, ex. today morning, this evening
                intent_dict['list_date'] = [datetime.now().strftime("%Y-%m-%d")]
                #print('a')
            if len(intent_dict['list_time']) == 0:  # the patient ask for data in some day without providing the time, ex. last 3 days, last week
                intent_dict['list_time'] = ['08:00:00']
                #print('b')
            ############################# TODO: get data from S3 with values in keys of 'patient_id', 'list_date', 'list_time', 'vital_sign'
            vital_sign_df = pd.read_csv('tmp/2024_03.csv') 
            #############################
            vital_signs_text = df_to_text(vital_sign_df, intent_dict, is_current=False)
            

            ########## PLOT ##########
    
            if intent_dict['is_plot']:
                # TODO: draw plot function ############################
                # use df directly to plot
                # save plot image and add the path to show_plot_list
                print('Plotted')
    print(intent_dict) #DEBUG    
    
    ################################
    ##########   IMAGES   ##########
    ################################
    
    if intent_dict['recognition'] or intent_dict['is_image']: # check if we need to get the vital sign
        ########## get real-time image from the EDGE DEVICE ##########
        if len(intent_dict['list_date']) == 0 and len(intent_dict['list_time']) == 0: 
            ############################# TODO: get image from S3
            image_path = ['tmp/HACER_mid_frame/putonglasses_angry_7_j.png']
            #############################
        
        ########## get real-time data from the AWS S3 ##########
        else: 
            ############################# TODO: get image from AWS S3
            image_path = ['tmp/HACER_mid_frame/putonglasses_angry_7_j.png']
            #############################
            
        ########## RECOGNITION ##########
        if intent_dict['recognition']:
            text_vision = "Classify the activities and emotions in this image."
            image_description = gpt(
                text = text_vision,
                model_name="gpt-4-vision-preview", 
                image_path = ['tmp/HACER_mid_frame/putonglasses_angry_7_j.png'], ############################# TODO: change image path  ############################
                system_prompt = system_prompt_vision,
                temperature = 0.2
            ) 
        
        
        if intent_dict['is_image']:
            ############################# TODO: add image path to show_image_list
            print('Showed image')
    
    ################################
    #########   ENDPOINT   #########
    ################################
    
    patient_info = patient_meta_df[patient_meta_df['patient_id']==int(patient_id)]
    
    text_endpoint = text_endpoint_format.format(current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                patient_id = patient_id,
                                name = patient_info['name'].values[0], 
                                sex = patient_info['sex'].values[0], 
                                address = patient_info['address'].values[0], 
                                phone = patient_info['phone'].values[0], 
                                dob = patient_info['birth'].values[0], 
                                age = patient_info['age'].values[0], 
                                image_description = image_description, 
                                vital_signs_data = vital_signs_text, 
                                question = doctor_question)
    print(text_endpoint) #DEBUG
    print()
    
    output = 'Done'
    #output = gpt(system_prompt=system_prompt_endpoint, text=text_endpoint, model_name="gpt-3.5-turbo", temperature=0.1)
    
    return output, show_list

#doctor_question = 'What is the current heart rate of the patient? 00001'
doctor_question = q51
response, show_list = NLP_engine(doctor_question)
print('==============')
print()
print(response) 

Plotted
{'patient_id': '00001', 'list_date': ['2024-03-14'], 'list_time': ['08:00:00', '08:30:00', '09:00:00', '09:30:00', '10:00:00', '10:30:00', '11:00:00', '11:30:00', '12:00:00', '12:30:00', '13:00:00', '13:30:00', '14:00:00'], 'vital_sign': ['heart_rate'], 'is_plot': True, 'recognition': False, 'is_image': False}

The current time is 2024-03-14 18:14:48.

Patient information:
ID Number: 00001
Name: Tammy Hale PhD
Sex: F
Address: 3364 Gallegos Cliff Apt. 995, New Hannahport, FL 27223
Caregiver phone number: 971561575483
Date of birth (yyyy-mm-dd): 1962-01-11
Age: 62

Image:
None

Vital signs:
Timestamp (Year-Month-Day Hour:Minute:Second), Heart Rate 
2024-03-14 08:00:00, 61 
2024-03-14 08:30:00, 77 
2024-03-14 09:00:00, 76 
2024-03-14 09:30:00, 80 
2024-03-14 10:00:00, 77 
2024-03-14 10:30:00, 93 
2024-03-14 11:00:00, 63 
2024-03-14 11:30:00, 98 
2024-03-14 12:00:00, 80 
2024-03-14 12:30:00, 62 
2024-03-14 13:00:00, 96 
2024-03-14 13:30:00, 71 
2024-03-14 14:00:00, 60 


Give me a 

In [339]:
q11 = "Give me the current blood pressure of the patient 00001"
q12 = "Give me the respository rate of the patient 00001 in yesterday afternoon"
q13 = "Provide me the heart rate of the patient 00001 in the last 3 days"

q21 = "Give me the current vital signs of the patient 00001"
q22 = "Give me the respository rate and heart rate of the patient 00001 from 8am to 2pm."
q23 = "Provide me the heart rate and temperature of the patient 00001 in the last 3 days"

q31 = "Show me the current image of the patient 00001"
q32 = "Give me the image of the patient 00001 in yesterday morning"
q33 = "Give me the image of the patient 00001 in the morning of the last 3 days"

q41 = "What is the patient 00001 doing and what is the patient's mood now?"
q42 = "What is the patient 00001 doing and what is the patient's mood at 2pm?"
q43 = "What is the patient 00001 doing and what is the patient's mood in the last three days?"

q51 = "Give me a plot of patient 00001 heart rate from 8am to 2pm."
q52 = "Give me a plot of patient 00001 oxygen saturation and body temperature from 8am to 2pm."
q53 = "Give me a plot of patient 00001 heart rate last week"

In [268]:
vital_sign = {
                "heart_rate": 61,
                "systolic_pressure": 99,
                "diastolic_pressure": 75,
                "respiratory_rate":	14,
                "body_temperature": 36.8,
                "oxygen_saturation": 99
            } 
vital_sign['heart_rate']

61

In [310]:
vital_sign_df = pd.read_csv('tmp/2024_03.csv') 
vital_sign_df

,time_stamp,heart_rate,systolic_pressure,diastolic_pressure,respiratory_rate,body_temperature,oxygen_saturation
0,2024-03-01 00:00:00,75,97,73,13,36.4,100
1,2024-03-01 00:30:00,77,104,75,14,36.4,97
2,2024-03-01 01:00:00,93,114,68,17,37.1,95
3,2024-03-01 01:30:00,100,104,80,17,36.9,96
4,2024-03-01 02:00:00,83,97,79,17,36.4,99
...,...,...,...,...,...,...,...
667,2024-03-14 21:30:00,90,110,74,16,36.7,97
668,2024-03-14 22:00:00,92,96,77,19,37.1,100
669,2024-03-14 22:30:00,76,111,78,13,36.9,98
670,2024-03-14 23:00:00,67,109,73,18,36.7,100


In [250]:
vital_sign_df = pd.read_csv('tmp/2024_03.csv')

intent_dict = {
    "patient_id": "unknown",
    "list_date": ["2024-03-12", "2024-03-13"],
    "list_time": ["06:00:00", "06:30:00", "07:00:00"],
    "vital_sign": ["heart_rate", "systolic_pressure", "diastolic_pressure"],
    "is_plot": True,
    "recognition": False,
    "is_image": False
}

# combine date and time
time_stamp_list = []
for date in intent_dict['list_date']:
    for time in intent_dict['list_time']:
        time_stamp = f'{date} {time}'
        time_stamp_list.append(time_stamp)

columns = ['time_stamp'] + intent_dict['vital_sign']
data_df = vital_sign_df[columns]
data_df = data_df[data_df['time_stamp'].isin(time_stamp_list)].reset_index()
print(data_df)
"""
vital_signs_text = ''
title_row = 'Timestamp (Year-Month-Day Hour:Minute:Second)'

vital_sign_var_to_text = {
    'heart_rate': 'Heart Rate', 
    'systolic_pressure': 'Systolic Pressure', 
    'diastolic_pressure': 'Diastolic Pressure', 
    'respiratory_rate': 'Respiratory Rate', 
    'body_temperature': 'Body Temperature', 
    'oxygen_saturation': 'Oxygen Saturation'
}

for vital_sign in intent_dict['vital_sign']:
    title_row += f', {vital_sign_var_to_text[vital_sign]}'
vital_signs_text += f'{title_row} \n'  
    
for i in range(len(data_df)):
    data_row_text = data_df.iloc[i]['time_stamp']
    
    for vital_sign in intent_dict['vital_sign']:
        data_row_text += f', {data_df.iloc[i][vital_sign]}'
    
    vital_signs_text += f'{data_row_text} \n'

print(vital_signs_text)    
#data_df"""

   index           time_stamp  heart_rate  systolic_pressure  \
0     12  2024-03-12 06:00:00          75                102   
1     13  2024-03-12 06:30:00          66                102   
2     14  2024-03-12 07:00:00          82                107   

   diastolic_pressure  
0                  72  
1                  74  
2                  63  


"\nvital_signs_text = ''\ntitle_row = 'Timestamp (Year-Month-Day Hour:Minute:Second)'\n\nvital_sign_var_to_text = {\n    'heart_rate': 'Heart Rate', \n    'systolic_pressure': 'Systolic Pressure', \n    'diastolic_pressure': 'Diastolic Pressure', \n    'respiratory_rate': 'Respiratory Rate', \n    'body_temperature': 'Body Temperature', \n    'oxygen_saturation': 'Oxygen Saturation'\n}\n\nfor vital_sign in intent_dict['vital_sign']:\n    title_row += f', {vital_sign_var_to_text[vital_sign]}'\nvital_signs_text += f'{title_row} \n'  \n    \nfor i in range(len(data_df)):\n    data_row_text = data_df.iloc[i]['time_stamp']\n    \n    for vital_sign in intent_dict['vital_sign']:\n        data_row_text += f', {data_df.iloc[i][vital_sign]}'\n    \n    vital_signs_text += f'{data_row_text} \n'\n\nprint(vital_signs_text)    \n#data_df"

In [175]:
doctor_question = "Give me the blood pressure in the last 3 days"
NLP_engine(doctor_question)

{'patient_id': 'unknown', 'list_date': ['2024-03-11', '2024-03-12', '2024-03-13'], 'list_time': [], 'vital_sign': ['systolic_pressure', 'diastolic_pressure'], 'is_plot': False, 'recognition': False, 'is_image': False}

False


I did not receive a valid patient ID number. Please provide a valid ID number. 001


False


I did not receive a valid patient ID number. Please provide a valid ID number. 00001


True
{'patient_id': '00001', 'list_date': ['2024-03-11', '2024-03-12', '2024-03-13'], 'list_time': [], 'vital_sign': ['systolic_pressure', 'diastolic_pressure'], 'is_plot': False, 'recognition': False, 'is_image': False}


In [50]:
system_prompt = """
Today is March 14, 2024.
You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object with the following keys:
1. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question. If the user is asking for data at the current moment, include the value 'current' in the list.
2. 'time': If the user wants to retrieve data for a specific time of day (hour and minute or periods like morning, afternoon, etc.), include this information as a list in this key.
3. 'vital_sign': A list of vital signs that the user is asking for.
4. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly or if the user has requested a plot.
6. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient."""
print(system_prompt)


Today is March 14, 2024.
You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object with the following keys:
1. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question. If the user is asking for data at the current moment, include the value 'current' in the list.
2. 'time': If the user wants to retrieve data for a specific time of day (hour and minute or periods like morning, afternoon, etc.), include this information as a list in this key.
3. 'vital_sign': A list of vital signs that the user is asking for.
4. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly or if the user has requested a plot.
6. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient.


In [ ]:
system_prompt = """
Your role is to analyze images captured by surveillance cameras to determine the activity being performed 
and the emotion displayed by any person in the image. You're equipped to recognize 9 activity classes: 
drink, put on glasses, put on jacket, read, sit down, stand up, take off glasses, take off jacket, and write, 
as well as 5 emotion classes: happy, neutral, sad, angry, and disgust. When an image is input, accurately output 
the activity and emotion class in the format (activity - emotion). If no person is visible in the image, respond 
with 'unidentifiable' for both activity and emotion. Similarly, if the person's face is not clear, especially in 
surveillance footage, output 'unidentifiable' for the emotion while still attempting to identify the activity. 
Provide classifications in a straightforward manner, without additional commentary or tone, just the class names."""